<a href="https://colab.research.google.com/github/sridatascience/inkersassignments/blob/master/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function  #import appropriate libraries 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()                 # 28 x 28 x 1 |  32x(3x3x1) = 288 parameters +32 parameters for bias
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -28x28x1 Output  28x28x32 (RF 1x1) (with padding, the i/p size becomes 30x30)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)#input - 28x28x32 |64x(3x3x32)=18432 parameters+64 parameters for bias Output 28x28x64 (RF 3x3) (with padding, the i/p size becomes 30x30)
        self.pool1 = nn.MaxPool2d(2, 2)             #input - 28x28x64 |MP(2x2) Output 14x14X64 (RF 6x6)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14x14x64 |128x(3x3x64)=73728 parameters+128 parameters for bias Output 14x14x128 (RF 8x8) (with padding, the i/p size becomes 16x16)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)#input - 14x14x128 |256x(3x3x128)=294912 parameters+256 parameters for bias Output 14x14x256 (RF 10x10) (with padding, the i/p size becomes 16x16)
        self.pool2 = nn.MaxPool2d(2, 2)               #input - 14x14x256|MP(2x2) Output 7x7x256 RF 20x20
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7x7X256|512x(3x3x256)=1179648 parameters+512 parameters for bias  Output 5x5x512 (RF 22x22)
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5x5x512|1024x(3x3x512)=4718592 parameters+ 1024 parameters for bias  Output 3x3x1024(RF 24x24 )          
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3x3x1024|10x(3x3x1024)=92160 parameters+ 10 parameters for bias   Ouptut 1x1x10  (RF 26x26)

    def forward(self, x):                               # forward propagation
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   # relu layer drops all negative values and propagates positive values as it is
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)                           # softmax layer to predict integers (0-9) based on a set probability

In [4]:
!pip install torchsummary    # Run the shell command from the script , it installs torchsummary package from shell
from torchsummary import summary
use_cuda = torch.cuda.is_available() # checks for the availability of GPU, if available returns True else False
# print(use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")  # Device stores "cuda" if GPU is available or 'cpu' if GPU is not available
model = Net().to(device)                # Moves the model to GPU
summary(model, input_size=(1, 28, 28))  # Input Image size is 28x28X1, 1 here specifies only 1 channel of the input
                                        # Input: width x height x num channels (depth)
                                        # Displays summary  of the model 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:


torch.manual_seed(1)         # similar to random_state=1 in machine learning to pick a set of sample data
batch_size = 128             #batch size of the images is 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,            # download the training image data , training  labels
                    transform=transforms.Compose([
                        transforms.ToTensor(),                      # change the images to tensors
                        transforms.Normalize((0.1307,), (0.3081,))   # normalize the tensors with mean and standard deviation
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([   # download the test image data , testing labels
                        transforms.ToTensor(),                               # change the images to tensors
                        transforms.Normalize((0.1307,), (0.3081,))           # normalize the tensors with mean and standard deviation
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
from tqdm import tqdm                    # Library Used to display the download progress bar of images
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.0075, Accuracy: 5837/10000 (58%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.9559, Accuracy: 5929/10000 (59%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.7460, Accuracy: 6822/10000 (68%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.7487, Accuracy: 6808/10000 (68%)



loss=0.6487144827842712 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.43it/s]



Test set: Average loss: 0.7451, Accuracy: 6822/10000 (68%)



In [1]:
from tqdm import tqdm 
from time import sleep
for i in tqdm(range(int(9e6))): 
    pass

100%|██████████| 9000000/9000000 [00:01<00:00, 4891773.63it/s]
